In [25]:
# Import necessary libraries
from sklearn.linear_model import Ridge, LinearRegression, LogisticRegression
from sklearn.metrics import accuracy_score,log_loss,mean_absolute_error
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import GridSearchCV,train_test_split ,StratifiedKFold,KFold
from xgboost import XGBClassifier,XGBRFRegressor
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.tree import DecisionTreeClassifier,DecisionTreeRegressor,ExtraTreeClassifier
from sklearn.ensemble import VotingClassifier, BaggingClassifier,BaggingRegressor,RandomForestClassifier,AdaBoostClassifier,GradientBoostingClassifier
from sklearn.neighbors import *
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
import os

warnings.simplefilter('ignore')

os.chdir('D:\kaggla_comp\hardness')

train = pd.read_csv('train.csv',index_col=0)
train

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
id,,,,,,,,,,,,
0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8


In [18]:
x= train.drop('Hardness',axis=1)
y=train['Hardness']
print(len(y))

10407


# XgBOOST

In [36]:
# Create an instance of the XGBoost Random Forest Regressor
xgbm = XGBRFRegressor(random_state=24)

# Create a K-Fold cross-validation object
# This will split the data into 5 folds, without preserving the class distribution
# 'shuffle=True' ensures the data is shuffled before splitting
# 'random_state=24' sets the random seed for reproducibility
kfold = KFold(n_splits=5, shuffle=True, random_state=24)

# Define the hyperparameter grid to search over
# 'n_estimators' controls the number of trees in the ensemble
# 'learning_rate' controls the step size of the XGBoost algorithm
# 'max_depth' controls the maximum depth of the individual trees
params = {'n_estimators': [5, 100, 150], 'learning_rate': [0.01, 0.1, 0.3, 0.5, 0.7], 'max_depth': [1, 3, 5, 7]}

# Create a GridSearchCV object
# This will perform a grid search to find the best combination of hyperparameters
# 'cv=kfold' uses the K-Fold cross-validation object
# 'verbose=3' sets the verbosity level to 3, which will print progress updates
# 'scoring='neg_median_absolute_error'' sets the scoring metric to negative median absolute error (lower is better)
gcv = GridSearchCV(xgbm, param_grid=params, cv=kfold, verbose=3, scoring='neg_median_absolute_error')

# Fit the GridSearchCV object to the data
# This will perform the grid search and find the best hyperparameters
gcv.fit(x, y)

# Print the best parameters found by the GridSearchCV
print("Best Parameters:", gcv.best_params_)

# Print the best score (i.e., the lowest negative median absolute error) found during the grid search
print("Best Score:", gcv.best_score_)

# Get the best model from the GridSearchCV object
best_model = gcv.best_estimator_

Fitting 5 folds for each of 60 candidates, totalling 300 fits
[CV 1/5] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=-1.341 total time=   0.0s
[CV 2/5] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=-1.324 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=-1.357 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=-1.367 total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=1, n_estimators=5;, score=-1.353 total time=   0.0s
[CV 1/5] END learning_rate=0.01, max_depth=1, n_estimators=100;, score=-1.340 total time=   0.0s
[CV 2/5] END learning_rate=0.01, max_depth=1, n_estimators=100;, score=-1.324 total time=   0.0s
[CV 3/5] END learning_rate=0.01, max_depth=1, n_estimators=100;, score=-1.357 total time=   0.0s
[CV 4/5] END learning_rate=0.01, max_depth=1, n_estimators=100;, score=-1.367 total time=   0.0s
[CV 5/5] END learning_rate=0.01, max_depth=1, n_estimators=100;, score=-1.3

# complete code later

In [37]:
best_model = gcv.best_estimator_

# Load the test data into a DataFrame named "test"
test = pd.read_csv("test.csv", index_col=0)

pred_hard = best_model.predict(test)

# Load the sample submission file into a DataFrame named "submit"
submit = pd.read_csv("sample_submission.csv")

# Replace the predicted hardness values in the submission file
submit['Hardness'] = pred_hard

# Save the submission file with the predicted hardness values
submit.to_csv("D:\kaggla_comp\hardness\sbt_rg_26_jun.csv", index=False)